## This script will allow us to re-order nanopore reads based on methylation status of specific CpGs

### Edwin Neumann
### 12/31/2023

In [1]:
# import packages
import numpy as np
import pandas as pd
import pysam
import argparse

parser = argparse.ArgumentParser()
parser.add_argument('-e', '--extracted_CpGs')
parser.add_argument('-oo', '--output_order')
parser.add_argument('-i', '--input_bam')
parser.add_argument('-ob', '--output_bam')
parser.add_argument('-start', '--start_coordinate')
parser.add_argument('-end', '--end_coordinate')
args = parser.parse_args()

In [2]:
# import the modkit extract file with read-specific modified base calls
#extracted_CpGs = pd.read_csv('./experiments/231216_Prnp_KRABless/mouse_980-1/20231216_1354_MN44120_FAY08737_953dd6cb/fast5_pass/pass/extracted.tsv', sep='\t')
extracted_CpGs = pd.read_csv(args.extracted_CpGs, sep='\t')
extracted_CpGs.head()

,read_id,forward_read_position,ref_position,chrom,mod_strand,ref_strand,ref_mod_strand,fw_soft_clipped_start,fw_soft_clipped_end,read_length,mod_qual,mod_code,base_qual,ref_kmer,query_kmer,canonical_base,modified_primary_base
0,53cdf38a-cbe7-4d90-974c-c537447fe542,9,-1,NC_000068.8,+,+,+,28,3,4967,0.056641,m,27,.,TTCGT,C,C
1,53cdf38a-cbe7-4d90-974c-c537447fe542,19,-1,NC_000068.8,+,+,+,28,3,4967,0.001953,m,16,.,TACGT,C,C
2,53cdf38a-cbe7-4d90-974c-c537447fe542,106,131749415,NC_000068.8,+,+,+,28,3,4967,0.919922,m,19,CACGT,CACGT,C,C
3,53cdf38a-cbe7-4d90-974c-c537447fe542,126,131749435,NC_000068.8,+,+,+,28,3,4967,0.001953,m,20,GACAG,GACGA,C,C
4,53cdf38a-cbe7-4d90-974c-c537447fe542,301,131749612,NC_000068.8,+,+,+,28,3,4967,0.810547,m,13,TCCGT,CTCGT,C,C


In [3]:
# define the window we want to consider (genomic coordinates)
#start = 131751500
#end = 131752500
start = args.start_coordinate
end = args.end_coordinate

# define minimum base quality to consider
qual_min = 9

In [4]:
# filter extracted_CpGs to just the region of interest and base_qual above threshold
filter_mask = (extracted_CpGs['ref_position'].between(start, end)) & (extracted_CpGs['base_qual'] >= 9)
extracted_CpGs_filtered = extracted_CpGs[filter_mask]
extracted_CpGs_filtered.head()

,read_id,forward_read_position,ref_position,chrom,mod_strand,ref_strand,ref_mod_strand,fw_soft_clipped_start,fw_soft_clipped_end,read_length,mod_qual,mod_code,base_qual,ref_kmer,query_kmer,canonical_base,modified_primary_base
28,53cdf38a-cbe7-4d90-974c-c537447fe542,2176,131751520,NC_000068.8,+,+,+,28,3,4967,0.826172,m,17,GTCGA,GTCGA,C,C
30,53cdf38a-cbe7-4d90-974c-c537447fe542,2216,131751560,NC_000068.8,+,+,+,28,3,4967,0.001953,m,23,AACGA,AACGA,C,C
31,53cdf38a-cbe7-4d90-974c-c537447fe542,2228,131751572,NC_000068.8,+,+,+,28,3,4967,0.998047,m,24,CTCGC,CTCGC,C,C
32,53cdf38a-cbe7-4d90-974c-c537447fe542,2233,131751577,NC_000068.8,+,+,+,28,3,4967,0.869141,m,26,TCCGT,TCCGT,C,C
34,53cdf38a-cbe7-4d90-974c-c537447fe542,2332,131751676,NC_000068.8,+,+,+,28,3,4967,0.994141,m,21,AGCGT,AGCGT,C,C


In [5]:
# calculate the fraction CpGs methylated for each read
# Group by 'read_id' and perform the calculations
CpGs_calculated = extracted_CpGs_filtered.groupby('read_id').apply(lambda group: pd.Series({
    'fraction_modified': (group['mod_qual'] >= 0.7).mean()
}))

# Reset the index to make 'read_id' a regular column
CpGs_calculated.reset_index(inplace=True)

# now sort the reads by fraction modified, highest to lowest
CpGs_calculated = CpGs_calculated.sort_values(by='fraction_modified', ascending=False).reset_index(drop=True)
CpGs_calculated.head()

,read_id,fraction_modified
0,4736bbe5-9b76-43f2-92f9-a8e37688fcac,0.906977
1,885d97c5-bc6b-410f-9076-7808aac091a6,0.897959
2,f8a15d36-5919-47eb-b73b-a1814c6ccc0e,0.830508
3,30f16671-628b-49dc-b375-572154cdf4a1,0.820000
4,ec212ae3-2577-4b50-86ac-446594949dc1,0.813953


In [6]:
# now save the txt file of the ordered reads
#CpGs_calculated['read_id'].to_csv('./experiments/231216_Prnp_KRABless/mouse_980-1/20231216_1354_MN44120_FAY08737_953dd6cb/fast5_pass/pass/reads_to_keep_ordered.txt', header=False, index=False)
CpGs_calculated['read_id'].to_csv(args.output_order, header=False, index=False)
#print("File 'reads_to_keep_ordered.txt' saved.")

File 'reads_to_keep_ordered.txt' saved.


In [8]:
# rename the reads in the filtered bam file based on the order of reads_to_keep_ordered

def rename_reads(input_bam_filename, output_bam_filename, order_filename):
    # Open the input BAM file for reading
    with pysam.AlignmentFile(input_bam_filename, 'rb') as input_bam:
        # Create a dictionary to map old read names to new names
        read_name_mapping = {}
        
        # Read the order from 'reads_to_keep_ordered.txt'
        with open(order_filename, 'r') as order_file:
            ordered_reads = [line.strip() for line in order_file.readlines()]
        
        # Assign new names starting from 1
        for new_name, old_name in enumerate(ordered_reads, start=100):
            read_name_mapping[old_name] = str(new_name)

        # Open the output BAM file for writing
        with pysam.AlignmentFile(output_bam_filename, 'wb', header=input_bam.header) as output_bam:
            # Iterate through the reads in the input BAM file
            for read in input_bam:
                # Check if the read name is in the mapping
                if read.query_name in read_name_mapping:
                    # Update the read name
                    read.query_name = read_name_mapping[read.query_name]
                    # Write the modified read to the output BAM file
                    output_bam.write(read)

# get file paths 
#order_filename = './experiments/231216_Prnp_KRABless/mouse_980-1/20231216_1354_MN44120_FAY08737_953dd6cb/fast5_pass/pass/reads_to_keep_ordered.txt'
#input_bam_filename = './experiments/231216_Prnp_KRABless/mouse_980-1/20231216_1354_MN44120_FAY08737_953dd6cb/fast5_pass/pass/filtered_output.bam'
#output_bam_filename = './experiments/231216_Prnp_KRABless/mouse_980-1/20231216_1354_MN44120_FAY08737_953dd6cb/fast5_pass/pass/filtered_ordered_output.bam'

# Replace 'input.bam', 'output.bam', and 'reads_to_keep_ordered.txt' with your actual file names
rename_reads(args.input_bam, args.output_bam, args.output_order)
